# Progression: Ordered Sequences for Workflows

**Progression** is an ordered sequence of UUIDs with Element identity. It combines:

- List-like operations: `append`, `extend`, `insert`, `remove`, `pop`
- Workflow operations: `move`, `swap`, `reverse`
- Idempotent set-like operations: `include`, `exclude`
- Query operations: `len`, `contains`, `getitem`, `index`

Progressions are primitives for state machine construction, representing execution order in workflow systems.

In [1]:
# Setup
from uuid import uuid4

from lionherd_core.base import Element, Progression

## 1. Construction & Append/Extend

Create progressions with UUIDs or Elements, then add items using `append` or `extend`.

In [ ]:
# Empty progression
prog = Progression(name="task_queue")
print(f"Empty: {prog}")

# With UUIDs
tasks = [uuid4() for _ in range(3)]
prog_tasks = Progression(order=tasks, name="pending")
print(f"\nWith UUIDs: {prog_tasks}")

# With Elements (auto-converts to UUIDs)
# NEW in PR #162: Normalization via @field_validator (no custom __init__)
elements = [Element() for _ in range(2)]
prog_elements = Progression(order=elements, name="active")
print(f"With Elements: {prog_elements}")

# Append and extend
prog.append(uuid4())
prog.extend([uuid4(), uuid4()])
print(f"\nAfter append/extend: {prog}, len={len(prog)}")

# NEW in PR #162: __bool__ protocol for empty checks
empty_prog = Progression()
print(f"\nEmpty progression bool: {bool(empty_prog)}")  # False
print(f"Non-empty progression bool: {bool(prog)}")  # True

In [3]:
# Performance comparison: extend() vs multiple append()
tasks = [uuid4() for _ in range(100)]

# ❌ Less efficient: Multiple operations
prog1 = Progression(name="inefficient")
for task in tasks:
    prog1.append(task)
print(f"Multiple append(): {len(prog1)} items added")

# ✅ More efficient: Single batch operation
prog2 = Progression(name="efficient")
prog2.extend(tasks)
print(f"Single extend(): {len(prog2)} items added")

print(f"\nBoth methods produce same result: {prog1.order == prog2.order}")
print("Use extend() for batch operations - cleaner and more efficient")

Multiple append(): 100 items added
Single extend(): 100 items added

Both methods produce same result: True
Use extend() for batch operations - cleaner and more efficient


### Batch Operations: extend() vs Multiple append()

The `extend()` method is optimized for adding multiple items at once. It's more efficient than calling `append()` in a loop because it performs a single batch operation instead of multiple individual operations.

## 2. Query Operations

Check length, membership, and access items by index.

In [4]:
uids = [uuid4() for _ in range(5)]
prog = Progression(order=uids, name="execution_order")

# Length and membership
print(f"Length: {len(prog)}")
print(f"First UUID in progression: {uids[0] in prog}")
print(f"Random UUID in progression: {uuid4() in prog}")

# Index access (supports negative indices)
print(f"\nFirst item: {prog[0]}")
print(f"Last item: {prog[-1]}")
print(f"Slice [1:3]: {prog[1:3]}")

# Find index
print(f"\nIndex of third UUID: {prog.index(uids[2])}")

# Iteration
print(f"\nAll items: {list(prog)}")

Length: 5
First UUID in progression: True
Random UUID in progression: False

First item: 556915c6-8c62-49a4-8523-1dc136e490b2
Last item: 2c9bc01a-acb6-4871-93c7-d65af7f42030
Slice [1:3]: [UUID('1a7f7fc1-62f4-4273-8e46-e12661d9f221'), UUID('f459ad27-9fe7-4477-8b58-0e71db34ac62')]

Index of third UUID: 2

All items: [UUID('556915c6-8c62-49a4-8523-1dc136e490b2'), UUID('1a7f7fc1-62f4-4273-8e46-e12661d9f221'), UUID('f459ad27-9fe7-4477-8b58-0e71db34ac62'), UUID('3769da52-a012-403d-a38c-8315242a11d2'), UUID('2c9bc01a-acb6-4871-93c7-d65af7f42030')]


In [5]:
# __list__() protocol for explicit conversion
prog = Progression(order=[uuid4() for _ in range(3)], name="workflow")

# Method 1: via __iter__ (implicit)
items_iter = list(prog)
print(f"Via __iter__: {len(items_iter)} items")

# Method 2: via __list__() (explicit protocol)
items_list = prog.__list__()
print(f"Via __list__(): {len(items_list)} items")
print(f"Both methods equivalent: {items_iter == items_list}")

# Use case: Snapshot for safe concurrent modification
original = Progression(order=[uuid4() for _ in range(5)])
snapshot = original.__list__()  # Explicit snapshot

# Modify original
original.pop()
original.pop()

print(f"\nOriginal after modification: {len(original)} items")
print(f"Snapshot preserved: {len(snapshot)} items")

Via __iter__: 3 items
Via __list__(): 3 items
Both methods equivalent: True

Original after modification: 3 items
Snapshot preserved: 5 items


## 2b. List Conversion with __list__()

Progression implements the `__list__()` protocol for explicit list conversion. While `list(prog)` works via `__iter__`, calling `__list__()` directly or using it in contexts requiring list protocol provides explicit conversion.

## 3. Workflow Operations: Move & Swap

Reorder items for priority scheduling or state transitions.

In [6]:
# Create progression representing task priority
task1, task2, task3, task4 = uuid4(), uuid4(), uuid4(), uuid4()
prog = Progression(order=[task1, task2, task3, task4], name="priority_queue")

print(f"Initial order: {[str(u)[:8] for u in prog.order]}")

# Move task3 (index 2) to front (index 0) - high priority
prog.move(2, 0)
print(f"After move(2, 0): {[str(u)[:8] for u in prog.order]}")

# Swap first and last tasks
prog.swap(0, -1)
print(f"After swap(0, -1): {[str(u)[:8] for u in prog.order]}")

# Reverse entire queue
prog.reverse()
print(f"After reverse(): {[str(u)[:8] for u in prog.order]}")

Initial order: ['ffa27f93', 'd92a276c', '87dd952d', '667bff10']
After move(2, 0): ['87dd952d', 'ffa27f93', 'd92a276c', '667bff10']
After swap(0, -1): ['667bff10', 'ffa27f93', 'd92a276c', '87dd952d']
After reverse(): ['87dd952d', 'd92a276c', 'ffa27f93', '667bff10']


## 4. Idempotent Operations: Include & Exclude

Set-like operations that are safe for retries and concurrent calls.

In [7]:
prog = Progression(name="task_registry")
task_id = uuid4()

# include: add if not present (idempotent)
print(f"First include: {prog.include(task_id)}")
print(f"Second include (duplicate): {prog.include(task_id)}")
print(f"Length after two includes: {len(prog)}")

# exclude: remove if present (idempotent)
print(f"\nFirst exclude: {prog.exclude(task_id)}")
print(f"Second exclude (absent): {prog.exclude(task_id)}")
print(f"Length after two excludes: {len(prog)}")

# Contrast with append (NOT idempotent)
prog.append(task_id)
prog.append(task_id)  # Creates duplicate
print(f"\nAfter two appends: len={len(prog)} (allows duplicates)")

First include: True
Second include (duplicate): False
Length after two includes: 1

First exclude: True
Second exclude (absent): False
Length after two excludes: 0

After two appends: len=2 (allows duplicates)


## 5. List Operations: Insert, Remove, Pop

Standard list operations with Element/UUID support.

In [8]:
uid1, uid2, uid3 = uuid4(), uuid4(), uuid4()
prog = Progression(order=[uid1, uid3], name="tasks")

# Insert at position
prog.insert(1, uid2)
print(f"After insert(1, uid2): {[str(u)[:8] for u in prog.order]}")

# Remove specific item
prog.remove(uid2)
print(f"After remove(uid2): {[str(u)[:8] for u in prog.order]}")

# Pop last item
popped = prog.pop()
print(f"Popped last: {str(popped)[:8]}")
print(f"Remaining: {[str(u)[:8] for u in prog.order]}")

# Pop first item (queue behavior)
prog.extend([uuid4(), uuid4()])
first = prog.popleft()
print(f"\nPopped first: {str(first)[:8]}")
print(f"After popleft: {len(prog)} items remaining")

After insert(1, uid2): ['d7bc50a5', 'ce9a6798', '07f907a4']
After remove(uid2): ['d7bc50a5', '07f907a4']
Popped last: 07f907a4
Remaining: ['d7bc50a5']

Popped first: d7bc50a5
After popleft: 2 items remaining


## 5b. Exception Handling & Safe Operations

Progression raises `NotFoundError` (not `IndexError`) for missing items, consistent with Pile/Graph/Flow semantic exceptions. The `pop()` method supports an optional `default` parameter for graceful fallback.

In [9]:
from lionherd_core.errors import NotFoundError

# Exception handling: pop() raises NotFoundError (not IndexError)
prog = Progression(order=[uuid4(), uuid4()])

try:
    prog.pop(10)  # Invalid index
except NotFoundError as e:
    print(f"NotFoundError caught: {e}")

# Safe fallback with default parameter (no exception)
result = prog.pop(10, default=None)
print(f"\nSafe pop with default: {result}")

# popleft() on empty also raises NotFoundError
empty = Progression()
try:
    empty.popleft()
except NotFoundError as e:
    print(f"\nEmpty popleft NotFoundError: {e}")

# Production pattern: safe queue processing
queue = Progression(order=[uuid4() for _ in range(3)], name="tasks")
processed = []

while (task := queue.pop(default=None)) is not None:
    processed.append(task)
    print(f"Processed: {str(task)[:8]}")

print(f"\nProcessed {len(processed)} tasks, queue empty: {len(queue) == 0}")

NotFoundError caught: Index 10 not found in progression of length 2

Safe pop with default: None

Empty popleft NotFoundError: Cannot pop from empty progression
Processed: cf9d542a
Processed: 8b87f077
Processed: fb6f0ef6

Processed 3 tasks, queue empty: True


## Summary

**Progression** provides ordered sequence management for workflows:

**Core Operations:**
- Construction: `Progression(order=[...], name="...")`
- List ops: `append`, `extend`, `insert`, `remove`, `pop`, `popleft`, `clear`
- Queries: `len`, `in`, `[index]`, `index()`, iteration

**Workflow Operations:**
- Reordering: `move(from, to)`, `swap(i, j)`, `reverse()`
- Idempotent: `include(item)`, `exclude(item)`

**New in PR #162:**
- `__init__` removed: All normalization via `@field_validator` (stricter validation)
- `NotFoundError` (not `IndexError`): Semantic consistency with Pile/Graph/Flow
  - `pop()` raises `NotFoundError` for invalid index
  - `popleft()` raises `NotFoundError` on empty
  - `_validate_index()` raises `NotFoundError` for out-of-bounds
- `pop(default=...)`: Safe fallback for graceful error handling
- `__bool__()` protocol: Empty progressions are falsy
- `__list__()`: Explicit list conversion protocol

**Key Properties:**
- Element inheritance (id, created_at, metadata)
- UUID or Element input (auto-converts via validator)
- Duplicates allowed (list semantics)
- Serialization support (JSON roundtrip)
- Workflow primitives for state machines

**Use Cases:**
- Task queues and execution order
- State machine construction (pending/active/completed)
- Priority scheduling with move/swap
- Idempotent registration with include/exclude

## 6. Serialization

Progressions inherit Element serialization with JSON support.

In [10]:
# Create progression
uids = [uuid4() for _ in range(3)]
original = Progression(order=uids, name="workflow_steps")

# Serialize to dict (JSON mode - UUIDs as strings)
data = original.to_dict(mode="json")
print("Serialized fields:")
print(f"  name: {data['name']}")
print(f"  order (first UUID): {data['order'][0][:8]}...")
print(f"  id: {data['id'][:8]}...")

# Deserialize (roundtrip)
restored = Progression.from_dict(data)
print(f"\nRestored: {restored}")
print(f"Order preserved: {restored.order == original.order}")
print(f"ID preserved: {restored.id == original.id}")

Serialized fields:
  name: workflow_steps
  order (first UUID): 5e76811b...
  id: 52db3940...

Restored: id=UUID('52db3940-3f30-4de6-80ff-bc60585968ac') created_at=datetime.datetime(2025, 11, 11, 19, 37, 43, 635966, tzinfo=datetime.timezone.utc) metadata={} name='workflow_steps' order=[UUID('5e76811b-d8ad-40a9-885d-30568d3278fc'), UUID('96aea431-17dc-4191-9d65-b08056bad353'), UUID('c0bfe411-008d-47d1-9360-324abc4d461c')]
Order preserved: True
ID preserved: True


## 7. Workflow State Machine Example

Progressions enable state machines for task execution tracking.

In [11]:
# State machine with three progressions
pending = Progression(name="pending")
active = Progression(name="active")
completed = Progression(name="completed")

# Initialize with tasks
tasks = [uuid4() for _ in range(5)]
pending.extend(tasks)

print(
    f"Initial state - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}"
)

# Start first task (pending → active)
task = pending.popleft()
active.append(task)
print(f"After start - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

# Complete task (active → completed)
task = active.pop()
completed.append(task)
print(
    f"After complete - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}"
)

# Retry failed task (active → pending)
pending.extend(active.order)
active.clear()
print(f"After retry - pending: {len(pending)}, active: {len(active)}, completed: {len(completed)}")

print(f"\nCompleted order preserves execution history: {[str(u)[:8] for u in completed.order]}")

Initial state - pending: 5, active: 0, completed: 0
After start - pending: 4, active: 1, completed: 0
After complete - pending: 4, active: 0, completed: 1
After retry - pending: 4, active: 0, completed: 1

Completed order preserves execution history: ['c0f0b169']


## Summary

**Progression** provides ordered sequence management for workflows:

**Core Operations:**
- Construction: `Progression(order=[...], name="...")`
- List ops: `append`, `extend`, `insert`, `remove`, `pop`, `popleft`, `clear`
- Queries: `len`, `in`, `[index]`, `index()`, iteration

**Workflow Operations:**
- Reordering: `move(from, to)`, `swap(i, j)`, `reverse()`
- Idempotent: `include(item)`, `exclude(item)`

**Key Properties:**
- Element inheritance (id, created_at, metadata)
- UUID or Element input (auto-converts)
- Duplicates allowed (list semantics)
- Serialization support (JSON roundtrip)
- Workflow primitives for state machines

**Use Cases:**
- Task queues and execution order
- State machine construction (pending/active/completed)
- Priority scheduling with move/swap
- Idempotent registration with include/exclude